# InfCtx trainer validation
This model being trained has the same settings as raven 1B5 model.
- Layer count: 24
- Embed size: 2048

The goal is to validate loss rate change, across the exact same hyper parameters
- 1024 data chunk size
- same learningrate / weightdecay / seed

With only the change in working context size
- 1024 context vs 128 context

> This project assumes you have the rwkv-exp conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Prepare the dataset

In [3]:
# First lets get the blank init model, these init model was generated
# using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
#
# As such I have preinitialized these blank models and uploaded them to HF for convinence
!mkdir -p ./model/
!rm -rf ./model/Echo-A-1B5-Init.pth
!cd ./model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
!ls -alh ./model/Echo-A-1B5-Init.pth

--2023-06-22 16:21:18--  https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
Resolving huggingface.co (huggingface.co)... 13.224.249.10, 13.224.249.119, 13.224.249.44, ...
Connecting to huggingface.co (huggingface.co)|13.224.249.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/cb/ef/cbef09abb2634a3375b28868bffa285226dfeabedec89b28c2fb302221164d66/0ec7214ed16737a6348254e6f96d8cdc04d3b5efbd5f53fe9337607ea42b5b9f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Echo-A-1B5-Init.pth%3B+filename%3D%22Echo-A-1B5-Init.pth%22%3B&Expires=1687681279&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2NiL2VmL2NiZWYwOWFiYjI2MzRhMzM3NWIyODg2OGJmZmEyODUyMjZkZmVhYmVkZWM4OWIyOGMyZmIzMDIyMjExNjRkNjYvMGVjNzIxNGVkMTY3MzdhNjM0ODI1NGU2Zjk2ZDhjZGMwNGQzYjVlZmJkNWY1M2ZlOTMzNzYwN2VhNDJiNWI5Zj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb

In [4]:
# Lets ensure some of the directories we need are setup
!mkdir -p ./datapaths/
!mkdir -p ./checkpoint/

In [3]:
# Lets preload the requried dataset
!cd ./RWKV-v4neo && python3 preload_dataset.py ./Mini-Enwiki-Test-A.yaml

Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 59.10it/s]
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-3d43d1724bef83d7_*_of_00016.arrow
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-5033407f38c97f24.arrow
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-78e7f3a5f1679aa4_*_of_00016.arrow

## Training Context Size 1024

In [15]:
# Lets start the training process
!cd ./RWKV-v4neo && python new_train.py fit -c ./Mini-Enwiki-Test-A.yaml

Setting ds_accelerator to cuda (auto detect)
Global seed set to 3941088705
Global seed set to 3941088705
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_1024_bf16...
Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 730.46it/s]
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a2364

## Training Context Size 128

In [8]:
# Lets start the training process
!cd ./RWKV-v4neo && python new_train.py fit -c ./Mini-Enwiki-Test-B.yaml

Setting ds_accelerator to cuda (auto detect)
Global seed set to 3941088705
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230624_104122-aenwikm09
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Mini-Enwiki-Test-B (lr 3e-4/1e-5/5, wd 0.01, ctx 128, data 1024, fixed seed)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/aenwikm09
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_128_bf16/build.ninja...
Building extension module wkv_128_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment varia